In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline

In [16]:
df = pd.read_csv('C:\\Users\\USER\\Desktop\\FYP\\fyp-sandbox-2\\Data\\Reg_Model_Training/model_training_data.csv')
df

,caseid,age_yrs,age_norm,age_bin_enc,gender_enc,origin_country_enc,wt_kg
0,18690315,9.00,0.499722,4,1,0,23.00
1,18690334,18.00,1.000000,5,0,1,61.80
2,18690432,14.00,0.777654,5,0,3,57.00
3,18690568,1.00,0.055031,2,1,1,10.00
4,18690589,0.25,0.013341,1,0,1,11.00
...,...,...,...,...,...,...,...
39650,24795513,16.00,0.888827,5,0,1,65.00
39651,24798894,12.00,0.666481,4,1,1,41.73
39652,24799324,16.00,0.888827,5,1,1,78.93
39653,24799665,0.17,0.008894,1,0,1,4.95


## Using the weights to train the RF model

In [17]:
#Splitting
X = df[['age_norm', 'age_bin_enc', 'gender_enc', 'origin_country_enc']]
y = df['wt_kg']

X.shape, y.shape

((39655, 4), (39655,))

In [18]:
#Splitting data
from sklearn.model_selection import train_test_split

X_trian, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_trian.shape, X_test.shape, y_train.shape, y_test.shape

((27758, 4), (11897, 4), (27758,), (11897,))

In [19]:
train_df = pd.DataFrame(X_trian)
train_df.head()

,age_norm,age_bin_enc,gender_enc,origin_country_enc
19039,0.888827,5,0,1
33112,0.666481,4,1,1
23288,0.499722,4,1,1
4829,1.000000,5,1,1
8472,0.555309,4,0,2


In [20]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.utils.class_weight import compute_sample_weight

# Compute sample weights based on age_bin_enc
sample_weights = compute_sample_weight(class_weight="balanced", y=train_df["age_bin_enc"])
sample_weights

array([0.39751962, 0.57901544, 0.57901544, ..., 0.39751962, 0.57901544,
       0.39751962], shape=(27758,))

In [21]:
#Training the model with weights
reg = RandomForestRegressor(n_estimators=100, random_state=7)
reg.fit(X_trian, y_train, sample_weight=sample_weights)

RandomForestRegressor(random_state=7)

In [22]:
preds = reg.predict(X_test)
pd.DataFrame({'Actual': y_test, 'Prediction': preds, 'Diff': abs(y_test-preds)})

,Actual,Prediction,Diff
5005,59.00,52.890353,6.109647
35666,13.00,11.422380,1.577620
23475,63.49,60.882321,2.607679
31749,33.00,37.408029,4.408029
34607,50.34,60.882321,10.542321
...,...,...,...
38609,59.00,57.680512,1.319488
2788,79.00,54.546618,24.453382
16029,27.22,31.162040,3.942040
31993,16.50,16.905105,0.405105


In [23]:
from sklearn.metrics import r2_score, root_mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

r2 = r2_score(y_test, preds)
rmse = root_mean_squared_error(y_test, preds)
mae = mean_absolute_error(y_test, preds)
mape = mean_absolute_percentage_error(y_test, preds)

#Displaying in a dataframe
all_fea = pd.DataFrame({'R2_Score': [r2], 'RMSE': [rmse], 'MAE': [mae], 'MAPE': [mape]})
all_fea

,R2_Score,RMSE,MAE,MAPE
0,0.863017,7.425207,5.455877,0.149739


In [24]:
import pickle

with open("C:\\Users\\USER\\Desktop\\FYP\\fyp-sandbox-2\\Scripts\\Demographic Scripts\\demog_wt_model/rf_new.pkl", "wb") as f:
    pickle.dump(reg, f)